<div align="center"> <h1 align="center"> Lego Freak's Wet Dream: An Interview at Denmark's Jewel </h1> </div>

I’m a Lego freak. This goes back as far as I can remember. My mom would tell me that I would sit with the Lego sets, following instructions, in complete silence, for hours on end. No food, no bathroom break, no reaction to questions. I remember the feeling of being entirely engrossed in the process, tunnel-zoned in on building. In fact, I believe that it is the last time I felt a true flow without even knowing it.

![IMG_0684](IMG_0684.jpeg)
<figcaption align = "center">Fig. 1: Me building Lego in Oct 1994.</figcaption>

So I recently applied to work as a data analyst intern at the famous Lego Group in Denmark. As part of the job interview process, I received the following take-home assignment:

You are asked to use the provided dataset and database schema to **understand the popularity of different Lego sets and themes**. The idea is to become familiarized with the data to be ready for an interview with a business stakeholder.

Create a report to summarize your findings. Include:

1. What is the average number of Lego sets released per year?
2. What is the average number of Lego parts per year?
3. Create a visualization for item 2.
4. What are the 5 most popular colors used in Lego parts?
5. [Optional] What proportion of Lego parts are transparent?
6. [Optional] What are the 5 rarest lego bricks?
7. Summarize your findings.

## Database Schema

![erd](data/lego_erd.png)

## Data

**inventory_parts**
- "inventory_id" - id of the inventory the part is in (as in the inventories table)
- "part_num" - unique id for the part (as in the parts table)
- "color_id" - id of the color
- "quantity" - the number of copies of the part included in the set
- "is_spare" - whether or not it is a spare part

**parts**
- "part_num" - unique id for the part (as in the inventory_parts table)
- "name" - name of the part
- "part_cat_id" - part category id (as in part_catagories table)

**part_categories**
- "id" - part category id (as in parts table)
- "name" - name of the category the part belongs to

**colors**
- "id" - id of the color (as in inventory_parts table)
- "name" - color name
- "rgb" - rgb code of the color
- "is_trans" - whether or not the part is transparent/translucent

**inventories**
- "id" - id of the inventory the part is in (as in the inventory_sets and inventory_parts tables)
- "version" - version number
- "set_num" - set number (as in sets table)

**inventory_sets**
- "inventory_id" - id of the inventory the part is in (as in the inventories table)
- "set_num" - set number (as in sets table)
- "quantity" - the quantity of sets included

**sets**
- "set_num" - unique set id (as in inventory_sets and inventories tables)
- "name" - the name of the set
- "year" - the year the set was published
- "theme_id" - the id of the theme the set belongs to (as in themes table)
- num-parts - the number of parts in the set

**themes**
- "id" - the id of the theme (as in the sets table)
- "name" - the name of the theme
- "parent_id" - the id of the larger theme, if there is one


***Acknowledgments**: Rebrickable.com*

## Key Takeaways from the Data Exploration

 - There are 11,673 different sets; 11,684 when we include different versions of sets; 11,723 when we include theme (according to set_num). Thus, there are different versions of the same set and some sets belong to more than one theme, of which there are 614. Although having a unique ID, some sets share names.
 - The time period of 2050-2017 spans 68 years but 2 are missing.
 - While average number of parts is approx. 162, median is around 45. Smaller sets are favored.
 - There are only 2,848 sets in the inventory. Each set only once.
 - There are 133 different colors. Colors differ in the level of darkness, presence of glitters, speckles, glow, transparency, chromatic effect, metallicity, and perl-like tint.
 - There are 25,779 unique parts with non-null part number.

## What is the average number of Lego sets released per year?

The number of sets per year shows the slow start of the Lego Group. Until 1975, 25 years after its first sets, the number of sets per year had hovered around 50. I nbetween 1975 and 1995, the sales increased to around 100 sets a year. Then came the explosion into high hundreds. The most prolific year appears to be 2014, with 713 sets released.

In [1]:
SELECT
    year,
    COUNT(set_num) AS num_sets_per_year
FROM sets
GROUP BY year
ORDER BY year ASC;

,year,num_sets_per_year
0,1950,7
1,1953,4
2,1954,14
3,1955,28
4,1956,12
...,...,...
61,2013,593
62,2014,713
63,2015,665
64,2016,596


In [2]:
SELECT
    ROUND(AVG(averages.num_sets_per_year), 0) AS average
FROM sets AS sa
JOIN
 (
   SELECT
       sb.year,
       COUNT(set_num) AS num_sets_per_year
   FROM sets AS sb
   WHERE num_parts > 0
   GROUP BY sb.year
 ) AS averages
ON sa.year = averages.year;

,average
0,356


For the average number of sets per year, sets were counted by their unique set number and sets where the number of parts was 0 or lower were excluded (mostly, these were booklets and manuals).

![average_per_year](average_per_year.png)
<figcaption align = "center">Fig. 3: Average Number of Sets Released per Year.</figcaption>

## What is the average number of Lego parts per year?

The number of lego parts steadily rises to approx. 35,000 parts per year in 2000. Since then the increase has become more rapid. It is important to note that all parts were included, irrespective of whether they are unique or spare. Therefore, it is a good indicator of how large the sets were throughout the years. For example, the most prolific year appears to be 2016 with 155,072 parts per 596 sets, which means there were fewer sets but larger. 

In [3]:
SELECT
    s.year,
    SUM(ip.quantity) AS number_of_parts_per_year
FROM sets AS s
FULL JOIN inventories
USING (set_num)
FULL JOIN inventory_parts AS ip
ON inventories.id = ip.inventory_id
WHERE ip.color_id NOT IN (-1, 9999)
GROUP BY s.year
ORDER BY s.year ASC;

,year,number_of_parts_per_year
0,1950,71
1,1953,66
2,1954,173
3,1955,1037
4,1956,221
...,...,...
61,2013,110574
62,2014,124872
63,2015,137804
64,2016,155072


But does it tell us anything about the creative potential? Let's look at the number of unique non-spare parts of different colors produced per year. We can see that the production of unique new parts is increasing almost each year.

In [4]:
SELECT
    s.year,
    COUNT(DISTINCT ip.part_num) AS number_of_unique_parts_per_year
FROM sets AS s
FULL JOIN inventories
USING (set_num)
FULL JOIN inventory_parts AS ip
ON inventories.id = ip.inventory_id
WHERE ip.is_spare = False AND ip.color_id NOT IN (-1, 9999)
GROUP BY s.year
ORDER BY s.year ASC;

,year,number_of_unique_parts_per_year
0,1950,6
1,1953,6
2,1954,27
3,1955,84
4,1956,46
...,...,...
61,2013,3619
62,2014,3672
63,2015,4127
64,2016,4217


Lego Group's success is partly based on the ability to bring new designs that, of course, require new parts. Here is how they are doing on average per year.

In [5]:
SELECT 
    ROUND(AVG(averages_nonspare.number_of_unique_parts_per_year), 0) AS average
FROM sets AS sa
FULL JOIN inventories
USING (set_num)
FULL JOIN inventory_parts AS ipa
ON ipa.inventory_id = inventories.id
JOIN
 (
    SELECT
        sb.year,
        COUNT(DISTINCT ipb.part_num) AS number_of_unique_parts_per_year
    FROM sets AS sb
    FULL JOIN inventories
    USING (set_num)
    FULL JOIN inventory_parts AS ipb
    ON inventories.id = ipb.inventory_id
    WHERE ipb.is_spare = False AND ipb.color_id NOT IN (-1, 9999)
    GROUP BY sb.year
 ) AS averages_nonspare
ON sa.year = averages_nonspare.year;

,average
0,2507


![avg_parts_per_year](avg_parts_per_year.png)
<figcaption align = "center">Fig. 6: Average Number of Unique Non-Spare Parts Released per Year.</figcaption>

## What are the 5 most popular colors used in Lego parts?

The five most popular colors by count of parts that are colored that way are displayed below. None of them are translucent. They encompass the most frequent colors used in Lego parts.

In [6]:
SELECT c.name, c.is_trans, SUM(ip.quantity) AS sum_quantity
FROM inventory_parts AS ip
FULL JOIN colors as c
ON ip.color_id = c.id
WHERE ip.quantity IS NOT NULL OR ip.is_spare IS NOT NULL
GROUP BY c.name, c.is_trans, ip.is_spare
ORDER BY sum_quantity DESC
LIMIT 5;

,name,is_trans,sum_quantity
0,Black,False,390863
1,White,False,218931
2,Light Bluish Gray,False,197244
3,Red,False,168029
4,Dark Bluish Gray,False,146603


![colors](colors.png)
<figcaption align = "center">Fig. 7: Five most popular colors.</figcaption>

## What proportion of Lego parts are transparent?

Continuing to work with unique, non-spare parts, slightly more than 1/20 of them is transparent.

In [23]:
SELECT COUNT(DISTINCT parts.part_num)::float * 100 / (SELECT COUNT(DISTINCT parts.part_num)::float FROM parts WHERE parts.part_num IS NOT NULL) AS proportion
FROM parts
FULL JOIN inventory_parts
USING (part_num)
FULL JOIN colors           
ON inventory_parts.color_id = colors.id
WHERE colors.is_trans = True AND inventory_parts.is_spare = False AND parts.part_num IS NOT NULL;

,proportion
0,5.593814


![percentage_translucent](percentage_translucent.png)
<figcaption align = "center">Fig. 8: Percentage of Unique, Non-Spare Parts that are Transparent.</figcaption>

## What are the 5 rarest lego bricks?

Rarest parts seem to stem from specialized themed sets and mostly include specific body parts, in particular torsos and heads.

In [22]:
SELECT parts.name AS part_name, COUNT(parts.part_num) AS count_parts
FROM parts
FULL JOIN inventory_parts
USING (part_num)
FULL JOIN colors           
ON inventory_parts.color_id = colors.id
WHERE inventory_parts.is_spare = False AND inventory_parts.color_id NOT IN (-1, 9999) AND parts.part_num IS NOT NULL
GROUP BY parts.name
ORDER BY count_parts ASC
LIMIT 5;

,part_name,count_parts
0,Torso SW Aayla Secura Print / Medium Blue Arm ...,1
1,DUPLO BALLOON,1
2,Torso Plain / Dark Gray Arms / Sand Green Hands,1
3,"Minifig Head with Furrowed Brow Lines, Black M...",1
4,"Minifig Head Black Eyebrows, Cheek Lines, Whit...",1


![rarest_parts](rarest_parts.png)
<figcaption align = "center">Fig. 9: Five rarest parts.</figcaption>

## Summary

The analysis focused on understanding the popularity of different Lego sets and themes by answering key questions about the average number of sets and parts released per year, the most popular colors used in Lego parts, the proportion of transparent parts, and the 5 rarest Lego bricks. The results showed that there is an average of 356 sets released per year, with an average of 2,507 unique non-spare parts per set. The 5 most popular colors in Lego parts are black, white, light bluish gray, red, and dark bluish gray, in that order. Furthermore, 5.6% of unique non-spare Lego parts are transparent. The 5 rarest Lego bricks are two torsos, two heads, and Duplo Baloons, all from specialized themed sets.

As a lifelong Lego fan, I was beyond thrilled to have the opportunity to prepare this analysis for an interview with the Lego Group. I have cherished playing with Lego sets since I was a child, and I wanted to use this analysis as a way to showcase my passion and knowledge for the brand. 

![IMG_0675](IMG_0675.jpeg)
<figcaption align = "center">Fig. 10: My Apollo 11 Lunar Lander in 2023.</figcaption>

I was determined to provide a comprehensive understanding of the popularity of different Lego sets and themes, delving into key questions such as the average number of sets and parts released each year, the most popular colors used in Lego parts, the proportion of transparent parts, and the 5 rarest Lego bricks. I am proud of the results of this analysis and I hope it demonstrates my expertise in data analysis, as well as my deep love for Lego. I would be honored to bring my passion and skills to the Lego Group and contribute to the continued success of the brand.